# Articulation points

See [wikipedia](https://en.wikipedia.org/wiki/Biconnected_component)

See algorithm at [link](https://stepik.org/lesson/12342/step/10?unit=2794)

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import itertools
import sys
import random
import threading
from random import randint
from collections import defaultdict, deque

In [2]:
# Disable plot axes by default
import matplotlib as mpl
if True:
    mpl.rc('axes.spines', top=False, bottom=False, left=False, right=False)
    mpl.rc('xtick', top=False, bottom=False, labelsize=0)
    mpl.rc('ytick', left=False, right=False, labelsize=0)

In [3]:
from IPython.display import Image, HTML

def pydot_image(graph, prog='dot', width=None):
    dot = nx.nx_pydot.to_pydot(graph)
    if width:
        return Image(dot.create_png(prog=prog), width=width)
    else:
        return HTML(dot.create_svg(prog=prog).decode())

In [4]:
from time import clock

prof_list = []

def prof_init():
    del prof_list[:]

def prof_this(func):
    def prof_wrap(*a, **kw):
        t0 = clock()
        ret = func(*a, **kw)
        prof_list.append(clock() - t0)
        return ret
    return prof_wrap

def prof_time():
    if not len(prof_list):
        prof_list.append(0.)
    return sum(prof_list) / len(prof_list)

In [5]:
test_edges = [tuple(map(int, s.split())) for s in '0 1\n1 2\n2 0\n3 2\n4 3\n4 2\n5 4\n'.splitlines()]

In [6]:
def make_random_graph(max_nodes, min_nodes=4, max_edges=None, max_node_edges=4):
    max_nodes = max(max_nodes, min_nodes)
    while True:
        num = randint(min_nodes, max_nodes)
        deg_in = [randint(1, max_node_edges) for i in range(num)]
        deg_out = deg_in[:]
        random.shuffle(deg_out)
        graph = nx.directed_configuration_model(in_degree_sequence=deg_in,
                                                out_degree_sequence=deg_out)
        edges = set()
        for a,b,c in list(graph.edges):
            if a != b:
                edges.add((a,b) if a < b else (b,a))
        if not edges:
            continue
        if max_edges and len(edges) > max_edges:
            continue
        graph = nx.Graph(list(edges))
        if len(graph) and nx.is_connected(graph):
            nodes = sorted(graph.nodes)
            graph = nx.relabel_nodes(graph, {nodes[i]: i for i in range(len(nodes))})
            return sorted(graph.edges)

In [7]:
def nx_articulation_points(edges):
    return sorted(nx.articulation_points(nx.from_edgelist(edges)))

print(' '.join(str(v) for v in nx_articulation_points(test_edges)))

2 4


## Prepare tests

In [8]:
all_tests = [make_random_graph(min_nodes=4, max_nodes=100) for _ in range(5000)]
print(max(list(map(len, all_tests))))

273


In [9]:
num_nodes = 6000
num_graphs = 5
max_edges = 20000
big_tests = [make_random_graph(min_nodes=num_nodes, max_nodes=num_nodes+10, max_edges=max_edges)
             for _ in range(num_graphs)]
print(max(list(map(len, big_tests))))

15047


## Recursive

In [10]:
@prof_this
def my_recursive_articulation_points(edge_list):
    adj = defaultdict(set)
    for a,b in edge_list:
        adj[a].add(b)
        adj[b].add(a)
    adj = [sorted(adj[v]) for v in sorted(adj.keys())]
    if not adj:
        return []
    #nv = max(a[-1] if a else 0 for a in adj) + 1
    nv = max(a[-1] for a in adj) + 1
    assert len(adj) == nv

    visited = [False] * nv
    k_no = [0]
    k_val = [None] * nv
    parents = [None] * nv
    edges = set()
    cuts = set()

    def dfs(v):
        visited[v] = True
        k_no[0] += 1
        k_val[v] = k_cur = l_cur = k_no[0]
        l_children = num_children = 0

        for u in adj[v]:
            if visited[u]:
                continue
            parents[u] = v
            edges.add((u,v))
            edges.add((v,u))
            num_children += 1
            l_child = dfs(u)
            if l_child < l_cur:
                l_cur = l_child
            if l_child > l_children:
                l_children = l_child

        z = parents[v]
        while z is not None:
            if z in adj[v] and (z,v) not in edges:
                k_z = k_val[z]
                if k_z < l_cur:
                    l_cur = k_z
            z = parents[z]

        if v == 0:
            if num_children > 1:
                cuts.add(v)
        else:
            if l_children >= k_cur:
                cuts.add(v)

        return l_cur

    dfs(0)

    return sorted(cuts)

edges = test_edges
#edges = list(tuple(map(int, s.split())) for s in sys.stdin)
cuts = my_recursive_articulation_points(edges)
print(' '.join(str(v) for v in cuts))

2 4


In [11]:
prof_init()
for test in all_tests:
    nx_cuts = nx_articulation_points(test)
    my_cuts = my_recursive_articulation_points(test)
    assert my_cuts == nx_cuts, 'not same!'
print('OK %.3g' % prof_time())

OK 0.00237


### IMPORTANT! IMPORTANT! IMPORTANT!

It's not enough to set recursion limit on Windows.
You've got to increase stack size for your function as well.
Since we can't set stack size for the main process, we use a workaround.
We increase threading stack size and run our function in a thread:

    threading.stack_size(64Mb)
    sys.setrecurstionlimit(10**5)
    global_result = [None]
    def wrapper(*args):
        global_result[0] = recursive_func(*args)
    thread = threading.Thread(target=wrapper, args=args)
    thread.start()
    thread.join()
    result = global_result[0]

In [18]:
threading.stack_size(64*1024*1024)
sys.setrecursionlimit(100100)

wrapper_result = [None]
def my_articulation_points_wrapper(test):
    wrapper_result[0] = None
    wrapper_result[0] = my_recursive_articulation_points(test)

prof_init()
for test in big_tests:
    nx_cuts = nx_articulation_points(test)
    thread = threading.Thread(target=my_articulation_points_wrapper, args=[test])
    thread.start()
    thread.join()
    my_cuts = wrapper_result[0]
    assert my_cuts == nx_cuts, 'not same!'
print('OK %.3g' % prof_time())

OK 13.1


## Slow, Non-recursive

In [19]:
@prof_this
def my_nonrec_articulation_points(edge_list):
    adj = defaultdict(set)
    for a,b in edge_list:
        adj[a].add(b)
        adj[b].add(a)
    adj = [adj[v] for v in sorted(adj.keys())]
    if not adj:
        return []
    nv = len(adj)
    assert len(adj) == max(max(a) for a in adj) + 1

    visited = [False] * nv
    k_no = [0]
    k_val = [None] * nv
    l_val = [None] * nv
    parents = [None] * nv
    l_children = [0] * nv
    n_children = [0] * nv
    edges = set()
    cuts = []

    k_val[0] = l_val[0] = 1
    visited[0] = True
    k_no = [1]
    todo1 = [list(adj[v]) for v in range(nv)]
    todo2 = [[] for v in range(nv)]
    stack = [0]
    v = 0

    while v is not None or stack:
        if v is None:
            v = stack[-1]
        while todo2[v]:
            u = todo2[v].pop()
            if l_val[u] < l_val[v]:
                l_val[v] = l_val[u]
            if l_val[u] > l_children[v]:
                l_children[v] = l_val[u]
        else:
            if todo1[v]:
                u = todo1[v].pop()
                if visited[u]:
                    continue
                parents[u] = v
                edges.add((v,u))
                n_children[v] += 1
                visited[u] = True
                k_no[0] += 1
                k_val[u] = l_val[u] = k_no[0]
                todo2[v].append(u)
                stack.append(u)
                v = u
            else:
                v = stack.pop()
                u = parents[v]
                while u is not None:
                    if (u in adj[v]) and k_val[u] < l_val[v] \
                            and ((u,v) not in edges) and ((v,u) not in edges):
                        l_val[v] = k_val[u]
                    u = parents[u]
                v = None

    for v in range(nv):
        if (v == 0 and n_children[v] > 1) or (v > 0 and l_children[v] >= k_val[v]):
            cuts.append(v)

    return sorted(cuts)

#edge_list = list(tuple(map(int, s.split())) for s in sys.stdin)
edge_list = test_edges
cuts = my_nonrec_articulation_points(edge_list)
print(' '.join(str(v) for v in cuts))


2 4


In [20]:
prof_init()
for test in all_tests:
    nx_cuts = nx_articulation_points(test)
    my_cuts = my_nonrec_articulation_points(test)
    assert my_cuts == nx_cuts, 'not same!'
print('OK %.3g' % prof_time())

OK 0.00252


In [21]:
prof_init()
for i, test in enumerate(big_tests):
    nx_cuts = nx_articulation_points(test)
    my_cuts = my_nonrec_articulation_points(test)
    assert my_cuts == nx_cuts, 'not same!'
print('OK %.3g' % prof_time())

OK 10.5


# !!!!!!!!!!!!!!!!!!!!!!!!!!!
## Fast, Iterative

[description of algorithm](https://stepik.org/lesson/12342/step/10)

In [143]:
@prof_this
def my_iter_articulation_points(edge_list, root=0, debug=False):
    from collections import defaultdict
    adj = defaultdict(set)
    for a,b in edge_list:
        assert a>=0 and b>=0
        adj[a].add(b)
        adj[b].add(a)
    adj = [sorted(adj[v]) for v in sorted(adj.keys())]
    nv = len(adj)
    if not nv:
        return []
    assert max(max(a) for a in adj) == nv-1

    parent = [-1] * nv
    visited = [False] * nv
    visited[root] = True
    parent[root] = root
    root_deg = 0

    kval = [None] * nv
    lval = [None] * nv
    kval[root] = lval[root] = 1
    cur_val = 2
    stack = [(root, iter(adj[root]))]
    edges = set()

    while stack:
        p, adj_p = stack[-1]
        for v in adj_p:
            if visited[v]:  continue
            visited[v] = True
            edges.add((v,p))
            edges.add((p,v))
            parent[v] = p
            kval[v] = lval[v] = cur_val
            cur_val += 1
            if p == root:
                root_deg += 1
            stack.append((v, iter(adj[v])))
            break
        else:
            stack.pop()

    for v in range(nv):
        l = lval[v]
        p = parent[v]
        while p != root:
            if p in adj[v] and (p,v) not in edges:
                if lval[p] < l:
                    l = lval[p]
            p = parent[p]
        lval[v] = l
        p = parent[v]
        if p != root and lval[p] > l:
            lval[p] = l

    cuts = set()
    for v in range(nv):
        l = lval[v]
        p = parent[v]
        while p != root:
            if l >= kval[p]:
                cuts.add(p)
            p = parent[p]

    if root_deg > 1:
        cuts.add(root)

    padj = [[] for v in range(nv)]
    for v in range(nv):
        p = v
        while p != root:
            p = parent[p]
            padj[v].append(p)
        
    if debug:
        print('elist ', sorted(edge_list))
        print('edges ', sorted(edges))
        print('adj   ', adj)
        print('p-adj ', padj)
        print('rdeg  ', root_deg)
        print('parent', parent)
        print('kval  ', kval)
        print('lval  ', lval)
        print('cuts  ', sorted(cuts))

    return sorted(cuts)

#edge_list = list(tuple(map(int, s.split())) for s in sys.stdin)
edge_list = test_edges
cuts = my_iter_articulation_points(edge_list, debug=True)
print(' '.join(str(v) for v in cuts))

elist  [(0, 1), (1, 2), (2, 0), (3, 2), (4, 2), (4, 3), (5, 4)]
edges  [(0, 1), (1, 0), (1, 2), (2, 1), (2, 3), (3, 2), (3, 4), (4, 3), (4, 5), (5, 4)]
adj    [[1, 2], [0, 2], [0, 1, 3, 4], [2, 4], [2, 3, 5], [4]]
p-adj  [[], [0], [1, 0], [2, 1, 0], [3, 2, 1, 0], [4, 3, 2, 1, 0]]
rdeg   1
parent [0, 0, 1, 2, 3, 4]
kval   [1, 2, 3, 4, 5, 6]
lval   [1, 2, 3, 3, 3, 6]
cuts   [1, 2, 3, 4]
1 2 3 4


In [109]:
prof_init()
for test in all_tests[:100]:
    nx_cuts = nx_articulation_points(test)
    my_cuts = my_iter_articulation_points(test)
    #assert my_cuts == nx_cuts, 'not same!'
print('OK %.3g' % prof_time())

OK 0.00464


In [111]:
prof_init()
for test in big_tests[:2]:
    nx_cuts = nx_articulation_points(test)
    my_cuts = my_iter_articulation_points(test)
    #assert my_cuts == nx_cuts, 'not same!'
print('OK %.3g' % prof_time())

OK 29.8


## From NetworkX

In [22]:
def _my_nx_dfs(adj, components=True):
    # depth-first search algorithm to generate articulation points and biconnected components
    visited = set()
    for start in sorted(adj.keys()):
        if start in visited:
            continue
        discovery = {start:0} # "time" of first discovery of node during search
        low = {start:0}
        root_children = 0
        visited.add(start)
        edge_stack = []
        stack = [(start, start, iter(adj[start]))]
        while stack:
            grandparent, parent, children = stack[-1]
            try:
                child = next(children)
                if grandparent == child:
                    continue
                if child in visited:
                    if discovery[child] <= discovery[parent]: # back edge
                        low[parent] = min(low[parent],discovery[child])
                        if components:
                            edge_stack.append((parent,child))
                else:
                    low[child] = discovery[child] = len(discovery)
                    visited.add(child)
                    stack.append((parent, child, iter(adj[child])))
                    if components:
                        edge_stack.append((parent,child))
            except StopIteration:
                stack.pop()
                if len(stack) > 1:
                    if low[parent] >= discovery[grandparent]:
                        if components:
                            ind = edge_stack.index((grandparent,parent))
                            yield edge_stack[ind:]
                            edge_stack=edge_stack[:ind]
                        else:
                            yield grandparent
                    low[grandparent] = min(low[parent], low[grandparent])
                elif stack: # length 1 so grandparent is root
                    root_children += 1
                    if components:
                        ind = edge_stack.index((grandparent,parent))
                        yield edge_stack[ind:]
        if not components:
            # root node is articulation point if it has more than 1 child
            if root_children > 1:
                yield start

@prof_this
def my_nx_articulation_points(edge_list):
    adj = defaultdict(set)
    for a,b in edge_list:
        adj[a].add(b)
        adj[b].add(a)
    if not adj:
        return []
    return sorted(set(_my_nx_dfs(adj, components=False)))

#edge_list = list(tuple(map(int, s.split())) for s in sys.stdin)
edge_list = test_edges
cuts = my_nx_articulation_points(edge_list)
print(' '.join(str(v) for v in cuts))

2 4


In [23]:
prof_init()
for test in all_tests:
    nx_cuts = nx_articulation_points(test)
    my_cuts = my_nx_articulation_points(test)
    assert my_cuts == nx_cuts, 'not same!'
print('OK %.3g' % prof_time())

OK 0.00115


In [24]:
prof_init()
for i, test in enumerate(big_tests):
    nx_cuts = nx_articulation_points(test)
    my_cuts = my_nx_articulation_points(test)
    assert my_cuts == nx_cuts, 'not same!'
print('OK %.3g' % prof_time())

OK 0.162


## Subprocess tests

In [25]:
@prof_this
def run_program(edges):
    from sys import executable
    from subprocess import run, STDOUT, PIPE
    sdata = ''.join('{} {}\n'.format(a,b) for a,b in edges)
    pname = 'articulation-points.py'
    res = run([executable, pname], stderr=STDOUT, stdout=PIPE, input=bytes(sdata,'utf8'))
    return res.stdout.strip().decode()

print(run_program(test_edges))

prof_init()
for test in all_tests[:100]:
    nx_cuts = ' '.join(str(v) for v in nx_articulation_points(test))
    my_cuts = run_program(test)
    assert my_cuts == nx_cuts, 'my:[{}] != nx:[{}]'.format(my_cuts, nx_cuts)
print('all OK %.3g' % prof_time())

prof_init()
for test in big_tests[:10]:
    nx_cuts = ' '.join(str(v) for v in nx_articulation_points(test))
    my_cuts = run_program(test)
    assert my_cuts == nx_cuts, 'not same!'
print('big OK %.3g' % prof_time())

2 4
all OK 0.208
big OK 0.519


## Debugging

In [26]:
if False:
    graph = nx.from_edgelist(test)
    dot = nx.nx_pydot.to_pydot(graph)
    HTML(dot.create_svg(prog='dot').decode())